# Get phase stability data

## Initialize required packages and data from _01-load-exp-data notebook_

In [ ]:
%run "core.ipynb"
analysis = load_analysis()


exp_data = analysis['exp_data']
phs_data = analysis['phs_data']
phs_mol_comp = analysis['phs_mol_comp']
phs_wt_comp = analysis['phs_wt_comp']
phs_symbols = analysis['phs_symbols'] 

## Load thermodynamic database

In [ ]:
modelDB = thermo.model.Database()
#phs = modelDB.get_phase('Grt'

## Initialize the phase stability experiment function 

### Initiates Phase Stability Data object to retrieve valid set of reactions from SVD for each experiment 

    * input:

        exp_data: 

        phs_wt_comp: 
        
        phase_symbols_key:
        
        modelDB:
        
        T_units:
        
        P_units

    * output:
        
        phase_stability_data: object
            
        


In [ ]:
phase_symbols_key = LEPR_phase_symbols

In [ ]:
phase_stability_data = thermo.calibrate.PhaseStabilityData(exp_data, phs_wt_comp, phase_symbols_key, modelDB,
                                                          T_units='C', P_units='GPa')

In [ ]:
# one of the things phase stability data does is filter out reactions invovling non-MELTS phases

In [ ]:
print('There are ' + str(len(phase_stability_data._phase_stability_exps)) + ' valid experiments')

In [ ]:
print('There are ' + str(len(phase_stability_data._exp_index_invalid)) + ' experiments with phases not found in the LEPR database')

In [ ]:
mask = [idx not in phase_stability_data._exp_index_invalid for idx in exp_data.index]


In [ ]:
rxn_1 = phase_stability_data._phase_stability_exps[0]

In [ ]:
rxn_1._phase_mol_endmem_comp

In [ ]:
mol_oxides1 = rxn_1._phase_mol_oxide_comp

In [ ]:
grt_mol_ox = mol_oxides1[-1]

In [ ]:
endmember_site_occ = np.array(
    [[3, 0, 0, 0, 2, 0, 0, 0, 0, 0],
     [0, 3, 0, 0, 2, 0, 0, 0, 0, 0],
     [0, 0, 3, 0, 2, 0, 0, 0, 0, 0],
     [0, 0, 0, 3, 2, 0, 0, 0, 0, 0],
     [3, 0, 0, 0, 0, 2, 0, 0, 0, 0],
     [0, 3, 0, 0, 0, 0, 1, 1, 0, 0],
     [3, 0, 0, 0, 0, 0, 0, 0, 1, 1]])

In [ ]:
endmember_stoic=[]

endmember_stoic.append(thermo.chem.format_mol_oxide_comp({'MgO':3, 'Al2O3':1, 'SiO2':3}))
endmember_stoic.append(thermo.chem.format_mol_oxide_comp({'CaO':3, 'Al2O3':1, 'SiO2':3}))
endmember_stoic.append(thermo.chem.format_mol_oxide_comp({'MnO':3, 'Al2O3':1, 'SiO2':3}))
endmember_stoic.append(thermo.chem.format_mol_oxide_comp({'FeO':3, 'Al2O3':1, 'SiO2':3}))
endmember_stoic.append(thermo.chem.format_mol_oxide_comp({'MgO':3, 'Cr2O3':1, 'SiO2':3}))
endmember_stoic.append(thermo.chem.format_mol_oxide_comp({'CaO':3, 'TiO2':1, 'FeO':1, 'SiO2':3}))
endmember_stoic.append(thermo.chem.format_mol_oxide_comp({'MgO':4, 'SiO2':4}))
endmember_stoic=np.array(endmember_comp)

In [ ]:
endmember_stoic

In [ ]:
mol_oxides = mol_oxides1[-1]

In [ ]:
from scipy import optimize

def mol_oxide_to_mol_endmember(mol_oxides, endmember_site_occ, endmember_stoic):
    
    endmember_site_occ_inv = np.linalg.pinv(endmember_site_occ.T)
    site_occ_stoic = np.dot(endmember_stoic.T, endmember_site_occ_inv)
    
    site_occ, residual = optimize.nnls(site_occ_stoic, mol_oxides)
    #endmember_comp = np.dot(endmember_site_occ_inv, site_occ)
    endmember_comp, residual = optimize.nnls(endmember_site_occ.T, site_occ)
    
    return endmember_comp, residual
    

In [ ]:
mol_oxide_to_mol_endmember(mol_oxides, endmember_site_occ, endmember_stoic)

In [ ]:
grt_mol_oxides = phs_mol_comp['Garnet']

In [ ]:
endmember_comp = []
residuals = []
for imol_ox in grt_mol_oxides.values:
    try:
        iendmember_comp, iresidual = mol_oxide_to_mol_endmember(imol_ox, endmember_site_occ, 
                                                                endmember_stoic)
    except:
        iendmember_comp = None
        iresidual = None
            
            
    endmember_comp.append(iendmember_comp)
    residuals.append(iresidual)
    

In [ ]:
ternary_frac = []
for iendmember_comp in endmember_comp:
    try:
        iternary_frac = np.sum(iendmember_comp[[0,1,3]])/np.sum(iendmember_comp)
        
    except:
        iternary_frac = np.nan
        
    ternary_frac.append(iternary_frac)
    
ternary_frac = np.array(ternary_frac)    

In [ ]:
endmember_comp[0]

In [ ]:
ternary_frac

# Reaction visualization

## rxn_svd visualization

In [ ]:
phs_info=phase_stability_data.get_phase_exp_info

In [ ]:
phs_info

In [ ]:
rxn_max = np.max(rxn_svd)

plt.figure()
plt.imshow(rxn_svd,cmap='seismic')
plt.clim(-rxn_max, +rxn_max)
plt.colorbar()

plt.xlabel('Endmember ID#')
plt.ylabel('Balanced Rxn ID#')
#plt.xlim(6.5,9.5)

### All reactions after filtering rxn_svd

In [ ]:
# not currently using this
plt.figure()
plt.imshow(rxn_coefs,cmap='seismic')
plt.clim(-rxn_max, +rxn_max)
plt.colorbar()

plt.xlabel('Endmember ID#')
plt.ylabel('Balanced Rxn ID#')

### Reaction correlation

In [ ]:
# not currently using this
ortho_project =np.dot(rxn_coefs, rxn_coefs.T) 

plt.figure()
plt.imshow(ortho_project, cmap='seismic')
plt.colorbar()
plt.clim(-1,1)

plt.xlabel('Basic Rxn ID#')
plt.ylabel('Basic Rxn ID#')
plt.title('Basic Rxn Correlation')

### Subset of new reactions 

In [ ]:
plt.figure()
plt.plot(rxn_coefs[::10].T, '-')
plt.xlabel('')
plt.ylabel('rxn')
plt.legend(['0','10','20','30'])

# Garnet endmember space

In [ ]:
iphs_name = 'Garnet'
iphs = modelDB.get_phase('Grt')

iendmem_names = iphs.endmember_names

imol_comp = phs_mol_comp[iphs_name]
endmem_labels = iphs.endmember_names


In [ ]:
imol_comp.T.head()

In [ ]:
iendmem_comp = []
method='intrinsic'

for idx in imol_comp.T:
    ijmol_comp = imol_comp.loc[idx].values
    
    ijendmem_comp = iphs.calc_endmember_comp(
        ijmol_comp, output_residual=False,  
        method=method, normalize=True)
    
    # iendmem_comp.append(ijendmem_comp/np.sum(ijendmem_comp))
    iendmem_comp.append(ijendmem_comp)
    
iendmem_comp = np.array(iendmem_comp)
    

In [ ]:
iendmem_comp

In [ ]:
iendmem_comp[:,0]

In [ ]:
tern_scatter(iendmem_comp[:,0], 
             iendmem_comp[:,1], 
             iendmem_comp[:,2], 
             0, fignum=None, marker='o', 
             vmin=-.1, vmax=+.1, cmap='viridis',
             labels=endmem_labels)

# Reaction affinity calculation

In [ ]:
affinities = phase_stability_data.calc_equil_rxn_affinities()

In [ ]:
affinities_all = np.hstack(affinities)
affinities_clean = affinities_all[~np.isnan(affinities_all)]

plt.figure()
plt.hist(affinities_clean, 101)
plt.figure()
plt.plot(ternary_frac[mask],rms_affinities,'ko')

In [ ]:
np.mean(rms_affinities>3250)

In [ ]:
affinities[16]

In [ ]:
rms_affinities = []
aff_max = []
for iaff in affinities:
    rms_affinities.append(np.sqrt(np.mean(iaff**2)))
    try:
        iaff_max = np.nanmax(np.abs(iaff))
    except:
        iaff_max = np.nan
    aff_max.append(iaff_max)
    
rms_affinities=np.array(rms_affinities)
aff_max = np.array(aff_max)
    

In [ ]:
aff_max

In [ ]:
import matplotlib.pyplot as plt
%matplotlib notebook



plt.figure()
#plt.semilogy(ternary_frac[mask], rms_affinities, 'bo')
plt.semilogy(ternary_frac[mask], aff_max, 'bo')


plt.ylabel('rms_affinities')
plt.xlabel('fraction in ternary garnet space')
plt.show()

In [ ]:
plt.figure()
plt.hist(ternary_frac[~np.isnan(ternary_frac)],70)

In [ ]:
aff1=affinities[1]

In [ ]:
rms = np.sqrt(np.mean(aff1**2))

In [ ]:
rms

In [ ]:
iexp = phase_stability_data._phase_stability_exps[100]
%timeit iexp.calc_equil_rxn_affinities()